<h1> Pred lemma using LSTM

In [1]:
from baseline_LSTM_LEM import *
import copy

In [2]:
#loading train data
sentences, lemmas = load_data("UD_French-Sequoia/fr_sequoia-ud-train.conllu")

In [3]:
# Build vocabulary
word_counts = Counter(word for sentence in sentences for word in sentence)
word_to_ix = {word: i + 1 for i, word in enumerate(word_counts)}  # +1 for padding
word_to_ix['<PAD>'] = 0
word_to_ix['<OOV>'] = len(word_to_ix)

lem_counts = Counter(lemma for lemma_list in lemmas for lemma in lemma_list)
lem_to_ix = {lemma: i + 1 for i, lemma in enumerate(lem_counts)}
lem_to_ix['<PAD>'] = 0

In [4]:
# Model parameters
embedding_dim = 64
hidden_dim = 128
lemmas_size = len(lem_to_ix)
epochs = 50
batch_size = 16

In [5]:
# Create dataset and data loader
dataset = LEMDataset(sentences, lemmas, word_to_ix, lem_to_ix)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [6]:
#define model, loss and optimizer
model = LEM_Model(len(word_to_ix), embedding_dim, hidden_dim, len(lem_to_ix))
loss_function = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

<h2> Training model

In [7]:
for epoch in range(epochs):
    total_loss = 0
    for sentence_in, targets in data_loader:
        model.zero_grad()
        lem_scores = model(sentence_in)
        loss = loss_function(lem_scores.view(-1, len(lem_to_ix)), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

Epoch 1, Loss: 3.7965178983552117
Epoch 2, Loss: 3.175051302569253
Epoch 3, Loss: 2.7333446289811816
Epoch 4, Loss: 2.5214447677135468
Epoch 5, Loss: 2.3127568679196493
Epoch 6, Loss: 2.244510797943388
Epoch 7, Loss: 2.167757431098393
Epoch 8, Loss: 2.081048342159816
Epoch 9, Loss: 2.0516748032399588
Epoch 10, Loss: 2.0107761131865636


In [8]:
loss, accuracy = evaluate_model(model, data_loader, loss_function, lem_to_ix)
print(f"Train Accuracy : {accuracy:.4f}")
print(f"Train loss : {loss:.4f}")

Train Accuracy : 0.7791
Train loss : 1.9689


<h2> Validation

In [9]:
sentences_dev, lemmas_dev = load_data("UD_French-Sequoia/fr_sequoia-ud-dev.conllu")

In [10]:
dataset_dev = LEMDataset(sentences_dev, lemmas_dev, word_to_ix, lem_to_ix)
data_loader_dev = DataLoader(dataset_dev, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [11]:
loss, accuracy = evaluate_model(model, data_loader_dev, loss_function, lem_to_ix)
print(f"Validation Accuracy : {accuracy:.4f}")
print(f"Validation loss : {loss:.4f}")

Validation Accuracy : 0.7854
Validation loss : 1.9054


<h2> Testing model

In [9]:
sentences_test, lemmas_test = load_data("UD_French-Sequoia/fr_sequoia-ud-test.conllu")

In [10]:
dataset_test = LEMDataset(sentences_test, lemmas_test, word_to_ix, lem_to_ix)
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

In [11]:
loss, accuracy = evaluate_model(model, data_loader_test, loss_function, lem_to_ix)
print(f"Test Accuracy : {accuracy:.4f}")
print(f"Test loss : {loss:.4f}")

Test Accuracy : 0.7935
Test loss : 1.9024
